# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [2]:
df=pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
df.duplicated().sum()

0

Датасет имеет 5 столбцов и 3214 строк. 
Пропуски отсутсвуют ,дубликаты отсутствуют,типы данных соответсвуют данным .

сalls — количество звонков

minutes — суммарная длительность звонков в минутах

messages — количество sms-сообщений

mb_used — израсходованный интернет-трафик в Мб

is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0)

In [6]:
df['is_ultra'].value_counts(normalize=True)

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

## Разбейте данные на выборки

Разобьем выборку по принципу 60/20/20.

In [7]:
df_train, df_valid_test=train_test_split(df,train_size=0.6,test_size=0.4,random_state=1234)
df_valid,df_test=train_test_split(df_valid_test,test_size=0.5,random_state=1234)

In [8]:
print('Размер тренериующей выборки:',df_train.shape[0])
print('Размер валидационной выборки:',df_valid.shape[0])
print('Размер тестовой выборки:',df_test.shape[0])

Размер тренериующей выборки: 1928
Размер валидационной выборки: 643
Размер тестовой выборки: 643


## Исследуйте модели

In [9]:
features_valid=df_valid.drop(['is_ultra'],axis=1) 
target_valid=df_valid['is_ultra']

In [10]:
features_train=df_train.drop(['is_ultra'],axis=1)
target_train=df_train['is_ultra']

In [11]:
features_test=df_test.drop(['is_ultra'],axis=1) 
target_test=df_test['is_ultra'] 

RANDOMTREECLASSIFIER

In [12]:
best_model=None
best_result=0
#best_depth=depth
for depth in range(1,22):
        model_tr=DecisionTreeClassifier(random_state=12345, max_depth=depth)
        model_tr.fit(features_train,target_train) 
        result=model_tr.score(features_valid,target_valid) 
        if result>best_result:
            best_model=model_tr
            best_result=result
            best_depth=depth    
print(f'результат:{best_result},глубина:{best_depth}')

результат:0.8289269051321928,глубина:6


Лучшее качество модели =0.83 при максимальной глубине 6

RANDOMFORESTCLASSIFIER



In [13]:
best_model=None
best_result=0
best_depth=depth

for estim in range(2,55,2):  
    for depth in range (3,66,3):
        model_frst=RandomForestClassifier(max_depth=depth,n_estimators=estim,random_state=12345) 
        model_frst.fit(features_train,target_train) 
        result=model_frst.score(features_valid,target_valid)

        if result>best_result:
            best_model=model_frst
            best_result=result
            best_depth=depth
print(f'результат: {best_result},глубина:{best_depth},количество деревьев:{estim}')      

результат: 0.8398133748055988,глубина:9,количество деревьев:54


Лучшее качество модели=0.84

LOGISTICREGRESSION

In [14]:
model_log=LogisticRegression()
model_log.fit(features_train,target_train) 
predict_Log=model_log.predict(features_valid)
print('accuracy:',accuracy_score(predict_Log,target_valid))

accuracy: 0.7589424572317263


Качесвто модели = 0.76

При обучении трех разных моделей получили лучшие метрики качества.

"Дерево предсказания" при гиперпараметрах max_depth= 6: 0.83

"Случайный лес" при гиперпараметрах max_depth=9, estim=54: 0.84

"Логистическая регрессия": 0.76

Лучшей модель:"Случайный лес" , эту модель будем проверять на тестовой выборке .



##  Проверьте модель на тестовой выборке

In [15]:
model_frst=RandomForestClassifier(max_depth=9,n_estimators=54,random_state=12345)
model_frst.fit(features_train,target_train)
prediction_frst=model_frst.predict(features_test)
print('n_estimators:',estim,end="")
print(' accuracy:',accuracy_score(prediction_frst,target_test))

n_estimators: 54 accuracy: 0.8149300155520995


Точность данной модели на тестовой выборке имеет близкую к валидационной , что означает отсутсвие проблем с переобучением модели.

## Вывод

Самые адекватные предсказания делает Random Forest, это видно по точности предсказания на валидационной и на тестовой выборке .

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
